In [6]:
import numpy as np 
import pandas as pd 
import os 
import random 
from sklearn.model_selection import train_test_split, StratifiedKFold
from tqdm import tqdm 
import torch 
import torch.nn as nn 
import time 
import datetime 
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor 
import json 
import ccxt 
from datetime import datetime, timedelta 
import seaborn as sns 
import pandas_ta as ta 
import matplotlib.pyplot as plt 
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score 

In [10]:
df = pd.read_csv("tokenpost_속보.csv") 
train_news = pd.read_csv("most_recent_btcusdt/train_finetune.csv")
test_news = pd.read_csv("most_recent_btcusdt/test_finetune.csv") 

def add_datetime(df):
    dates = df['dates'].values 
    years, months, days, hours = [], [], [], [] 
    for i in tqdm(range(len(dates)), position=0, leave=True):
        splitted = dates[i].split(' ') 
        d1 = splitted[0].split('-') 
        years.append(int(d1[0])) 
        months.append(int(d1[1])) 
        days.append(int(d1[2])) 
        t = splitted[2].split(":") 
        hours.append(int(t[0]))  
    df['year'] = years 
    df['month'] = months 
    df['day'] = days 
    df['hour'] = hours 
    df.rename(columns={'titles':'title', 'contents':'content'},inplace=True)
    return df 

df = add_datetime(df)
df.drop(columns={'dates'},inplace=True)
df.head(3)

100%|██████████| 78331/78331 [00:00<00:00, 437975.19it/s]


,title,content,year,month,day,hour
0,"바이든, 마이클 바 연준 부의장 지명... 전 리플 이사회 멤버",코인데스크에 따르면 바이든 미국 대통령이 미국 재무부 소속 공무원이자 전 리플(XR...,2022,4,16,0
1,스위스 취리히 공항에 대형 비트코인 광고판 등장,"암호화폐 전문 미디어 비트코인매거진에 따르면, 스위스 취리히 공항에 대형 비트코인 ...",2022,4,16,0
2,"BTC, 지난 5분간 1.18% 상승",BTC가 바이낸스 USDT 마켓 기준 지난 5분간 1.18% 상승했다. 현재 BTC...,2022,4,15,23


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("CryptoModel/cryptobert-base-all", use_auth_token="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")

model = AutoModelForSequenceClassification.from_pretrained("CryptoModel/cryptobert-base-all", use_auth_token="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", num_labels=3)

model.eval() 

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/245 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/667 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483M [00:00<?, ?B/s]

Some weights of the model checkpoint at CryptoModel/cryptobert-base-all were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [8]:
def bert_tokenizer(s1,s2,MAX_LEN=512): 
    encoded_dict = tokenizer.encode_plus(
        text = s1, 
        text_pair = s2, 
        add_special_tokens = True, 
        pad_to_max_length = True, 
        max_length = MAX_LEN, 
        return_attention_mask = True 
    )
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] 
    token_type_id = encoded_dict['token_type_ids'] 
    return input_id, attention_mask, token_type_id

In [ ]:
s1,s2 = df['title'].values[10], df['content'].values[10]

device = torch.device("cuda") 

model.to(device)
inputs = tokenizer(s1,s2, return_tensors="pt").to(device)
with torch.no_grad():
    logits = model(**inputs).logits

logits

In [16]:
# load dataframe 
import json

with open('BTC_USDT-4h-3.json') as f:
    d = json.load(f)
    
chart_df = pd.DataFrame(d)
chart_df = chart_df.rename(columns={0:"timestamp",
                                    1:"open",
                                    2:"high",
                                    3:"low",
                                    4:"close",
                                    5:"volume"})

def process(df): 
    binance = ccxt.binance() 
    dates = df['timestamp'].values 
    timestamp = [] 
    for i in range(len(dates)): 
        date_string = binance.iso8601(int(dates[i])) 
        date_string = date_string[:10] + " " + date_string[11:-5] 
        timestamp.append(date_string) 
    df['datetime'] = timestamp 
    df = df.drop(columns={'timestamp'})
    return df

chart_df = process(chart_df)

chart_df

,open,high,low,close,volume,datetime
0,4261.48,4349.99,4261.32,4349.99,82.088865,2017-08-17 04:00:00
1,4333.32,4485.39,4333.32,4427.30,63.619882,2017-08-17 08:00:00
2,4436.06,4485.39,4333.42,4352.34,174.562001,2017-08-17 12:00:00
3,4352.33,4354.84,4200.74,4325.23,225.109716,2017-08-17 16:00:00
4,4307.56,4369.69,4258.56,4285.08,249.769913,2017-08-17 20:00:00
...,...,...,...,...,...,...
10239,40585.54,40640.00,39320.02,39433.60,13727.261710,2022-04-22 12:00:00
10240,39433.60,39786.62,39177.00,39502.92,9189.603560,2022-04-22 16:00:00
10241,39502.92,39835.23,39401.00,39709.18,3716.510950,2022-04-22 20:00:00
10242,39709.19,39872.64,39406.25,39463.02,3424.421740,2022-04-23 00:00:00


In [14]:
chart_df

,open,high,low,close,volume,datetime,hour,day,month,year,Targets
datetime,,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99,82.088865,2017-08-17 04:00:00,4,17,8,2017,0.0
2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30,63.619882,2017-08-17 08:00:00,8,17,8,2017,0.0
2017-08-17 12:00:00,4436.06,4485.39,4333.42,4352.34,174.562001,2017-08-17 12:00:00,12,17,8,2017,1.0
2017-08-17 16:00:00,4352.33,4354.84,4200.74,4325.23,225.109716,2017-08-17 16:00:00,16,17,8,2017,1.0
2017-08-17 20:00:00,4307.56,4369.69,4258.56,4285.08,249.769913,2017-08-17 20:00:00,20,17,8,2017,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-22 12:00:00,40585.54,40640.00,39320.02,39433.60,13727.261710,2022-04-22 12:00:00,12,22,4,2022,1.0
2022-04-22 16:00:00,39433.60,39786.62,39177.00,39502.92,9189.603560,2022-04-22 16:00:00,16,22,4,2022,0.0
2022-04-22 20:00:00,39502.92,39835.23,39401.00,39709.18,3716.510950,2022-04-22 20:00:00,20,22,4,2022,0.0


In [96]:
# load dataframe 
import json

with open('BTC_USDT-4h-3.json') as f:
    d = json.load(f)
    
chart_df = pd.DataFrame(d)
chart_df = chart_df.rename(columns={0:"timestamp",
                                    1:"open",
                                    2:"high",
                                    3:"low",
                                    4:"close",
                                    5:"volume"})

def process(df): 
    binance = ccxt.binance() 
    dates = df['timestamp'].values 
    timestamp = [] 
    for i in range(len(dates)): 
        date_string = binance.iso8601(int(dates[i])) 
        date_string = date_string[:10] + " " + date_string[11:-5] 
        timestamp.append(date_string) 
    df['datetime'] = timestamp 
    df = df.drop(columns={'timestamp'})
    return df

chart_df = process(chart_df)

hours = []
days = [] 
months = [] 
years = [] 
for dt in tqdm(chart_df['datetime']):
    hour = pd.to_datetime(dt).hour 
    day = pd.to_datetime(dt).day 
    month = pd.to_datetime(dt).month 
    year = pd.to_datetime(dt).year 
    hours.append(hour) 
    days.append(day) 
    months.append(month)
    years.append(year) 

chart_df['hour'] = hours
chart_df['day'] = days 
chart_df['month'] = months 
chart_df['year'] = years 

targets = [] 
close = chart_df['open'].values 
high = chart_df['high'].values 
low = chart_df['low'].values 

threshold = 0.0075

for i in range(close.shape[0]-1):
    high_volatility = (high[i+1]-close[i]) / close[i] 
    low_volatility = (low[i+1]-close[i]) / close[i] 
    if high_volatility >= threshold:
        targets.append(0) 
    elif low_volatility <= -threshold:
        targets.append(1) 
    else:
        targets.append(2) # do not trade 

targets.append(None) 

chart_df['Targets'] = targets 
        
chart_df.set_index(pd.DatetimeIndex(chart_df["datetime"]), inplace=True)

print("=== Feature Engineering ===")
chart_df['ebsw'] = chart_df.ta.ebsw(lookahead=False)
chart_df['cmf'] = chart_df.ta.cmf(lookahead=False)
chart_df['vwap'] = chart_df.ta.vwap(lookahead=False)  
chart_df['vwap/open'] = chart_df['vwap'] / chart_df['open']
chart_df['high/low'] = chart_df['high'] / chart_df['low'] 
chart_df['close/open'] = chart_df['close'] / chart_df['open']
chart_df['high/open'] = chart_df['high'] / chart_df['open'] 
chart_df['low/open'] = chart_df['low'] / chart_df['open']

# differencing 
for l in range(1,6): 
    for col in ['open','high','low','close','volume']: 
        val = chart_df[col].values 
        val_ret = [None for _ in range(l)] 
        for i in range(l, len(val)):
            if val[i-l] == 0:
                ret = 1
            else:
                ret = val[i] / val[i-l] 
            val_ret.append(ret) 
        chart_df['{}_change_{}'.format(col, l)] = val_ret   


chart_df = chart_df.dropna()  
chart_df = chart_df.drop(columns={'datetime', 
                                  'open', 
                                  'high', 
                                  'low', 
                                  'close', 
                                  'volume', 
                                  'vwap'}) 

chart_df.head(2)

100%|██████████| 10244/10244 [00:04<00:00, 2061.79it/s]


=== Feature Engineering ===


,hour,day,month,year,Targets,ebsw,cmf,vwap/open,high/low,close/open,...,open_change_4,high_change_4,low_change_4,close_change_4,volume_change_4,open_change_5,high_change_5,low_change_5,close_change_5,volume_change_5
datetime,,,,,,,,,,,,,,,,,,,,,
2017-08-23 16:00:00,16,23,8,2017,1.0,0.00000,0.121501,0.980956,1.038033,0.974007,...,1.046272,1.031877,1.022327,1.004063,1.093820,1.060846,1.048760,1.040436,1.019077,1.406636
2017-08-23 20:00:00,20,23,8,2017,0.0,0.57735,0.097590,1.001464,1.026746,0.994568,...,1.012716,1.015253,1.008545,1.009603,1.075979,1.023881,1.012272,1.013929,1.003317,0.955893


In [97]:
train_size = int(chart_df.shape[0] * 0.95) 
train_df = chart_df.iloc[:train_size,:] 

val_size = int(chart_df.shape[0] * 0.01) 
val_df = chart_df.iloc[train_size:train_size+val_size,:] 

test_df = chart_df.iloc[train_size+val_size:,:] 

train_df.shape, val_df.shape, test_df.shape 

((9693, 37), (102, 37), (409, 37))

In [98]:
input_columns = [] 
for col in train_df.columns:
    if col != 'Targets' and col != 'year':
        input_columns.append(col)         

In [99]:
X_train = train_df[input_columns].values  
Y_train = train_df['Targets'].values

X_val = val_df[input_columns].values 
Y_val = val_df['Targets'].values 

X_test = test_df[input_columns].values
Y_test = test_df['Targets'].values



X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape  

((9693, 35), (9693,), (102, 35), (102,), (409, 35), (409,))

In [100]:
class_weights = compute_class_weight(class_weight = "balanced", 
                                     classes = np.unique(Y_train), 
                                     y = Y_train)

d = {0:class_weights[0], 1:class_weights[1], 2:class_weights[2]}
d

{0: 0.6463292658531706, 1: 0.9348958333333334, 2: 2.609854604200323}

In [101]:
clf = TabNetClassifier() 

clf.fit(X_train, 
        Y_train, 
        eval_set=[(X_val, Y_val)], 
        eval_metric=['logloss', 'balanced_accuracy'],
        weights = d, 
        max_epochs=150,
        patience=150)  

Device used : cuda
epoch 0  | loss: 1.17434 | val_0_logloss: 1.16684 | val_0_balanced_accuracy: 0.33453 |  0:00:00s
epoch 1  | loss: 1.01385 | val_0_logloss: 1.10393 | val_0_balanced_accuracy: 0.33652 |  0:00:01s
epoch 2  | loss: 0.88552 | val_0_logloss: 1.45336 | val_0_balanced_accuracy: 0.33333 |  0:00:02s
epoch 3  | loss: 0.7456  | val_0_logloss: 1.83435 | val_0_balanced_accuracy: 0.33333 |  0:00:02s
epoch 4  | loss: 0.71435 | val_0_logloss: 1.73198 | val_0_balanced_accuracy: 0.33333 |  0:00:03s
epoch 5  | loss: 0.68889 | val_0_logloss: 1.74969 | val_0_balanced_accuracy: 0.33333 |  0:00:04s
epoch 6  | loss: 0.67299 | val_0_logloss: 1.61855 | val_0_balanced_accuracy: 0.33333 |  0:00:04s
epoch 7  | loss: 0.67935 | val_0_logloss: 1.43538 | val_0_balanced_accuracy: 0.33333 |  0:00:05s
epoch 8  | loss: 0.6562  | val_0_logloss: 1.64176 | val_0_balanced_accuracy: 0.33333 |  0:00:06s
epoch 9  | loss: 0.65319 | val_0_logloss: 1.37067 | val_0_balanced_accuracy: 0.33333 |  0:00:06s
epoch 10 | 

epoch 85 | loss: 0.55796 | val_0_logloss: 0.63965 | val_0_balanced_accuracy: 0.66746 |  0:00:56s
epoch 86 | loss: 0.57173 | val_0_logloss: 0.66597 | val_0_balanced_accuracy: 0.62201 |  0:00:56s
epoch 87 | loss: 0.56514 | val_0_logloss: 0.6701  | val_0_balanced_accuracy: 0.65324 |  0:00:57s
epoch 88 | loss: 0.55217 | val_0_logloss: 0.65034 | val_0_balanced_accuracy: 0.65152 |  0:00:58s
epoch 89 | loss: 0.56539 | val_0_logloss: 0.65124 | val_0_balanced_accuracy: 0.58001 |  0:00:58s
epoch 90 | loss: 0.55859 | val_0_logloss: 0.64246 | val_0_balanced_accuracy: 0.58161 |  0:00:59s
epoch 91 | loss: 0.56154 | val_0_logloss: 0.62911 | val_0_balanced_accuracy: 0.60606 |  0:00:59s
epoch 92 | loss: 0.56253 | val_0_logloss: 0.65059 | val_0_balanced_accuracy: 0.68939 |  0:01:00s
epoch 93 | loss: 0.55983 | val_0_logloss: 0.68266 | val_0_balanced_accuracy: 0.71558 |  0:01:01s
epoch 94 | loss: 0.54851 | val_0_logloss: 0.66027 | val_0_balanced_accuracy: 0.63131 |  0:01:01s
epoch 95 | loss: 0.54927 | val

In [102]:
cnt = 0
pred = clf.predict(X_test)
for i in range(len(pred)):
    if Y_test[i] == float(pred[i]):
        cnt += 1 
        
cnt / len(pred) * 100

63.325183374083124

In [103]:
print(f1_score(Y_test, pred, average='macro'))
print(f1_score(Y_test, pred, average='micro'))
print(f1_score(Y_test, pred, average='weighted')) 

0.6147525076552174
0.6332518337408313
0.6702039196118003
